### Imports

In [2]:
import importlib
import os
from tqdm import tqdm
import traceback

import findatree.io as io
import findatree.transformations as transformations
import findatree.interactive as interactive
import findatree.geo_to_image as geo_to_image
import findatree.segmentation as segmentation
import findatree.photometric.shadow as shadow
import findatree.photometric.features as features

from bokeh.plotting import save
from bokeh.io import output_file

_ = importlib.reload(io)
_ = importlib.reload(transformations)
_ = importlib.reload(interactive)
_ = importlib.reload(geo_to_image)
_ = importlib.reload(segmentation)
_ = importlib.reload(shadow)
_ = importlib.reload(features)

### Definitions

In [3]:
# Define full paths to directories containing dsm, dtm, ortho and shape-files 
dir_names=[]
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DSMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DTMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Orthos'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Kronen'])

# Define directories where return hdf5s and htmls are stored
dir_name_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"
dir_name_html = r"C:\Data\lwf\processed\2020\html"

# Only process tnr numbers where a shape file is present
process_shapes_only = True

# Optionally process only specific tnrs, if empty all are processed
process_tnrs_only = [
#     6462,
#     10547,
#     11581,
#     11601,
#     12124,
#     12551,
]

#### Processing

#### Prepare main loop

In [4]:
importlib.reload(io)

# Get all available tnr numbers in directory
tnrs = io.find_all_tnrs_in_dir(dir_names[0])
print(f"Number of tnrs found: {len(tnrs)}")

if len(process_tnrs_only) > 0:
    tnrs = [tnr for tnr in tnrs if tnr in process_tnrs_only]
    
# Find all tnr numbers where a shape file is present
if process_shapes_only:
    tnrs = [tnr for tnr in tnrs if 'path_shapes' in io._find_paths_in_dirs(dir_names, tnr).keys()]
    print(f"Only processing {len(tnrs)} tnrs with shapes file")

Number of tnrs found: 261
Only processing 153 tnrs with shapes file


#### Run main loop

In [5]:
exception_traceback = []

for tnr in tqdm(tnrs):
    
    try:
        ################# [1] Load channels
        params_channels = {'tnr': tnr}
        channels, params_channels = geo_to_image.channels_load(dir_names, params_channels, verbose=False)
    
        ################# [2] Load human generated crowns. Compute photometric features.
        crowns_human, params_crowns_human = io.load_shapefile(dir_names, params_channels, verbose=False)
        features.crowns_add_features(
            channels,
            params_channels,
            crowns_human,
            params_crowns_human,
        )

        ################# [3] Generate and save watershed crowns
#         params_crowns_water = {}

#         crowns_water, params_crowns_water = segmentation.watershed(
#             channels,
#             params_channels,
#             params_crowns_water,
#             verbose=False,
#         )
#         io.crowns_to_hdf5(crowns_water, params_crowns_water, dir_name = dir_name_hdf5)

        ################# [4] Create bokeh plot and save as html
        plt = interactive.Plotter()
        plt.width = 400
        plt.channels_downscale = 0
        plt.show_features = [
            'id',
            'ba', 'kkl', 'sst', 'nbv',
            'perc25_ndvi','perc50_ndvi','perc75_ndvi',
        ]

        plt.add_channels(channels, params_channels)

        plt.figures_add_rgb(perc=2)
        plt.figures_add_rgb(perc=2, color_code='533', weight_code = '133')
        plt.figures_add_gray('mask')
        plt.figures_add_gray('chm')

        plt.togglers_add_crowns(crowns_human, params_crowns_human)

        # Create and save layout
        layout = plt.create_layout()
        output_file(
            filename = os.path.join(dir_name_html, f"tnr{tnr}.html"),
            title = f"tnr{tnr}_{transformations.current_datetime()}",
            mode='inline',
            )
        
        ################# [5] Save channels, crowns and layout
        io.channels_to_hdf5(channels, params_channels, dir_name = dir_name_hdf5)
        io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)
        save(layout)
        
    except:
        exception_traceback.append({
            'tnr': int(tnr),
            'traceback': traceback.format_exc(),
            })

# Save exceptions in .yaml
io.list_of_dicts_to_yaml(
    path = os.path.join(dir_name_hdf5, f"{transformations.current_datetime()}_batch_init_all.yaml"),
    list_of_dicts = exception_traceback,
    )

print()
print('Done!')
print(f"{len(exception_traceback)} exceptions occured")

100%|█████████████████████████████████████████████████████████| 153/153 [2:11:12<00:00, 51.45s/it]


Done!
0 exceptions occured


In [12]:
for e in exception_traceback:
    print(e['tnr'])
    print('-->', e['traceback'])

10545
--> Traceback (most recent call last):
  File "C:\Users\flori\AppData\Local\Temp\ipykernel_9636\1729340539.py", line 59, in <module>
    io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)
  File "C:\Repos\findatree\findatree\io.py", line 321, in crowns_to_hdf5
    assert features.shape[0] == params_crowns['number_crowns'], message
AssertionError: `len(crowns['features'][photometric]` is 33 but `params_crowns['number_crowns']` is 34)

